In [3]:
# Import
%matplotlib inline

import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd


%load_ext autoreload
%autoreload 2

In [4]:
# Open file
data_folder = './data/results/'
mylist = []
for chunk in pd.read_csv(data_folder + 'df.csv', sep=',', low_memory=False, chunksize=5000):
    mylist.append(chunk)
df = pd.concat(mylist, axis=0)
df.name = 'df'
del mylist
df = df[df.DSDECOD != np.nan]

In [5]:
#à supprimer quand data sera finalisé

df = df.replace({"\<":"", "\>":"", "\'":".", "\=":"", "-7-3":"-7.3", "-1-5":"-1.5", "-0-3":"-0.3", ".-3.9":"-3.9"}, regex=True).astype(float)

In [6]:
#get X and y
X_reg = df.loc[:, df.columns != 'DSDECOD'].to_numpy()
y_reg = df['DSDECOD'].to_numpy()

les propositions sont faites à partir de: https://scikit-learn.org/stable/modules/feature_selection.html

## 1. Univariate feature selection

Univariate feature selection works by selecting the best features based on univariate statistical tests. It can be seen as a preprocessing step to an estimator.
NEED NON Na DATA!

In [7]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [17]:
X = X_reg
y = y_reg
X_new = SelectKBest(chi2, k=2).fit_transform(X_reg, y_reg)
X_new.shape

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

## 2. Recursive feature elimination

A recursive feature elimination example with automatic tuning of the number of features selected with cross-validation. Need NON NAN values

In [8]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification

In [20]:
# Build a classification task using 3 informative features
X, y = X_reg, y_reg

# Create the RFE object and compute a cross-validated score.
svc = SVC(kernel="linear")
# The "accuracy" scoring shows the proportion of correct classifications

min_features_to_select = 1  # Minimum number of features to consider
rfecv = RFECV(
    estimator=svc,
    step=1,
    cv=StratifiedKFold(2),
    scoring="accuracy",
    min_features_to_select=min_features_to_select,
)
rfecv.fit(X, y)

print("Optimal number of features : %d" % rfecv.n_features_)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [1]:
# Plot number of features VS. cross-validation scores
plt.figure()
plt.xlabel("Number of features selected")
plt.ylabel("Cross validation score (accuracy)")
plt.plot(
    range(min_features_to_select, len(rfecv.grid_scores_) + min_features_to_select),
    rfecv.grid_scores_,
)
plt.show()

NameError: name 'plt' is not defined

## 3. Sequential Feature Selection

In [9]:
from sklearn.linear_model import RidgeCV
from sklearn.feature_selection import SelectFromModel

In [10]:
X, y = X_reg, y_reg

ridge = RidgeCV(alphas=np.logspace(-6, 6, num=5)).fit(X, y)
importance = np.abs(ridge.coef_)



threshold = np.sort(importance)[-3] + 0.01

sfm = SelectFromModel(ridge, threshold=threshold).fit(X, y)
print(f"Features selected by SelectFromModel: {feature_names[sfm.get_support()]}")
sfm = SelectFromModel(ridge, threshold=threshold).fit(X, y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

or

In [11]:
from sklearn.feature_selection import SequentialFeatureSelector

In [ ]:
sfs_forward = SequentialFeatureSelector(ridge, n_features_to_select=2, direction="forward").fit(X, y)

sfs_backward = SequentialFeatureSelector(ridge, n_features_to_select=2, direction="backward").fit(X, y)

print("Features selected by forward sequential selection: "f"{feature_names[sfs_forward.get_support()]}")

print("Features selected by backward sequential selection: "f"{feature_names[sfs_backward.get_support()]}")
